In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV
from RWN import RWN, get_rfweight

In [ ]:
boston = fetch_openml(name='boston', version=1)
x = boston.data
y = boston.target

In [ ]:
print("X的形状:", x.shape) 
print("y的形状:", y.shape) 

In [ ]:
y=np.array(y).reshape(-1, 1)# 最后一列作为目标变量
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x = scaler_x.fit_transform(x)
y = scaler_y.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size=0.2,              
    random_state=42,            
    shuffle=True,             
)

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

# 从训练集中划分验证集（用于超参数优化）


def target_function(hs, n_iter, lr, bs, tol, tau, n_estimators):
    
    
    # 准备数据（这里你需要确保 x_train1, y_train1, x_test1, y_test1 是全局可访问的或作为参数传入）
    rf = RandomForestRegressor(n_estimators=int(n_estimators))
                                     
                                    
    rf.fit(x_train, y_train)
    mrfw, mrfwn = get_rfweight(rf, x_train)
    
    # 设置 RWN3 模型的参数
    model_rwn = RWN( hs=int(hs), device="cuda")
    model_rwn.fit(x_train, y_train, mrfw, tau=tau, d=False, batch_size=int(bs), n_iter=int(n_iter), lr=float(lr), tol=float(tol), verbose=False)
    
    # 预测结果
    y_pred = model_rwn.predict(x_test)
    y_pred = y_pred.reshape(-1, 1)  
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    # 返回 -distance 作为优化目标（因为 BayesianOptimization 默认寻找最大值）
    return r2

# 更新超参数的搜索空间，删除 quantile
pbounds = {
    'hs': (64, 256),
    'n_iter': (500, 2000),
    'lr': (1e-4, 1e-3),
    'bs': (4, 256),
    'tol': (1e-6, 1e-4),
    'tau': (0.0, 1.0),
    'n_estimators':(50,200)
}

# 创建贝叶斯优化器对象
optimizer = BayesianOptimization(
    f=target_function,
    pbounds=pbounds,
    random_state=1
)

# 运行贝叶斯优化
optimizer.maximize(init_points=50, n_iter=10000)
best_params = optimizer.max['params']

# 输出最佳参数和对应的损失
print(optimizer.max)

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

# 从训练集中划分验证集（用于超参数优化）


def target_function(hs, n_iter, lr, bs, tol, tau, n_estimators):
    
    
    # 准备数据（这里你需要确保 x_train1, y_train1, x_test1, y_test1 是全局可访问的或作为参数传入）
    rf = RandomForestRegressor(n_estimators=int(n_estimators))
                                     
                                    
    rf.fit(x_train, y_train)
    mrfw, mrfwn = get_rfweight(rf, x_train)
    
    # 设置 RWN3 模型的参数
    model_rwn = RWN( hs=int(hs), device="cuda")
    model_rwn.fit(x_train, y_train, mrfw, tau=tau, d=False, batch_size=int(bs), n_iter=int(n_iter), lr=float(lr), tol=float(tol), verbose=False)
    
    # 预测结果
    y_pred = model_rwn.predict(x_test)
    y_pred = y_pred.reshape(-1, 1)  
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    # 返回 -distance 作为优化目标（因为 BayesianOptimization 默认寻找最大值）
    return r2

# 更新超参数的搜索空间，删除 quantile
pbounds = {
    'hs': (64, 256),
    'n_iter': (500, 2000),
    'lr': (1e-4, 1e-3),
    'bs': (4, 256),
    'tol': (1e-6, 1e-4),
    'tau': (0.0, 1.0),
    'n_estimators':(50,200)
}

# 创建贝叶斯优化器对象
optimizer = BayesianOptimization(
    f=target_function,
    pbounds=pbounds,
    random_state=1
)

# 运行贝叶斯优化
optimizer.maximize(init_points=50, n_iter=10000)
best_params = optimizer.max['params']

# 输出最佳参数和对应的损失
print(optimizer.max)

In [ ]:
rf = RandomForestRegressor(
        n_estimators=int(best_params['n_estimators']))
rf.fit(x_train, y_train)
mrfw, mrfwn = get_rfweight(rf, x_train)
    
    # --- RWN3模型训练 ---
model_rwn = RWN(
        hs=int(best_params['hs']),
        device="cuda"
    )
model_rwn.fit(
        x_train,
        y_train,
        mrfw,
        tau=best_params['tau'],
        d=False,
        batch_size=int(best_params['bs']),
        n_iter=int(best_params['n_iter']),
        lr=best_params['lr'],
        tol=best_params['tol'],
        verbose=False
    )
y_pred = model_rwn.predict(x_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")